In [ ]:
#import torch packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat,savemat
import copy

In [ ]:
#for docomo, 9 CUDAs are available in total, we only use CUDA 0 is this program
num_device = torch.cuda.device_count()
print('There are '+str(num_device)+' GPUs. We will use CUDA 0 in this program.')
device = torch.device('cuda:0')

In [ ]:
#assign parameter values
input_nodes=512
output_nodes=512
middle_nodes=512
test_rate=0.2
min_epoch=30
max_epoch=500
lr=1e-3
dropout_rate=0.05
num_BS=4
num_beam=512

In [ ]:
#load datasets from .mat files
in_set_file = loadmat('DLCB_Dataset/DLCB_input.mat')
in_set = in_set_file['DL_input']#in_set.shape=(54481,256)
out_set_file = loadmat('DLCB_Dataset/DLCB_output.mat')
out_set = out_set_file['DL_output']#out_set.shape=(54481,2048)

In [ ]:
#define a Network
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(input_nodes,middle_nodes)
        self.fc2 = nn.Linear(middle_nodes,middle_nodes)
        self.fc3 = nn.Linear(middle_nodes,middle_nodes)
        self.fc4 = nn.Linear(middle_nodes,middle_nodes)
        self.fc5 = nn.Linear(middle_nodes,output_nodes)
        
    def forward(self,x):
        # 4 hidden layers with each fc + relu + dropout
        x = x.to(device)
        x = x.view(-1,512)
        x = F.dropout(F.relu(self.fc1(x)),p = dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc2(x)),p = dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc3(x)),p = dropout_rate,training = True)
        x = F.dropout(F.relu(self.fc4(x)),p = dropout_rate,training = True)
        # 1 output layer with fc + relu
        x = F.relu(self.fc5(x))
        return x  
net = Net()
net = net.to(device)

#define Loss Function and Optimization method
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(net.parameters(),lr=lr)

In [ ]:
class DataSetGenerator(data.Dataset):
    def __init__(self,input_set,output_set):
        self.input_set = input_set
        self.output_set = output_set
    
    def __getitem__(self,index):
        x = torch.from_numpy(self.input_set[index])
        label = torch.from_numpy(self.output_set[index])
        return torch.FloatTensor(x),torch.FloatTensor(label)

    def __len__(self):
        return self.input_set.shape[0]

In [ ]:
#get num_total_user from in_set
def getTotalUser(in_set):
    return in_set.shape[0]
num_total_user = getTotalUser(in_set)
print(num_total_user)#num_total_user is an int

In [ ]:
#get DL_size_list, 17 percentages are fixed according to the author
def getDLSizeList():
    DL_pr_list = np.array([.001,.05,.1,.15,.2,.25,.3,.35,.4,.45,.5,.55,.6,.65,.7,.75,.8])
    return np.floor(DL_pr_list*num_total_user).astype(np.int64).tolist()
##DL_size_list = getDLSizeList()
##print(DL_size_list)

In [ ]:
#get num_train and num_test, both are lists according to DL_size_list, num_total_user, test_rate
def getAmount(DL_size_list,num_total_user,test_rate):
    num_train=np.floor(np.asarray(DL_size_list)*.8).astype(np.int64).tolist()
    num_test=int(num_total_user*test_rate)
    return num_train,num_test
##num_train,num_test = getAmount(DL_size_list,num_total_user,test_rate)
##print(num_train,num_test)

In [ ]:
#specifically for the i-th DL_size, return its in_train_pre,in_test_pre,out_train_pre,out_test_pre(all before any adjustments)
def eachInOutPre(i,num_train,num_test,num_total_user,in_set,out_set):
    train_index = np.random.choice(range(0,num_total_user),size=num_train[i],replace=False)
    rem_index = set(range(0,num_total_user))-set(train_index)
    test_index = list(set(np.random.choice(list(rem_index),size=num_test,replace=False)))
    in_train_pre = in_set[train_index]
    in_test_pre = in_set[test_index]
    out_train_pre = out_set[train_index]
    out_test_pre = out_set[test_index]
    return in_train_pre,out_train_pre,in_test_pre,out_test_pre,test_index
##in_train_pre,out_train_pre,in_test_pre,out_test_pre,test_index = eachInOutPre(16,num_train,num_test,num_total_user,in_set,out_set)
##print(in_train_pre.shape,out_train_pre.shape)

In [ ]:
#arrange input into real part and imaginary part one by one
def arrangeInput(lst):
    re=np.array(lst).real
    im=np.array(lst).imag
    return np.stack((re,im),axis=2).reshape(lst.shape[0],lst.shape[1]*2)
##in_train = arrangeInput(in_train_pre)
##in_test = arrangeInput(in_test_pre)
##print(in_train_pre.shape,in_train.shape)
##print(in_train_pre[12][85],in_train[12][85*2],in_train[12][85*2+1])

In [ ]:
#specificially for i BS(s) in usage, divide output columns by num_beam
def divideOutput(i,num_beam,lst):
    return np.array(lst)[:,i*num_beam:(i+1)*num_beam]
##out_train = divideOutput(2,num_beam,out_train_pre)
##out_test = divideOutput(2,num_beam,out_test_pre)
##print(out_train_pre.shape,out_train.shape)
##print(out_train_pre[12][85+1024],out_train[12][85])

In [ ]:
#for the i_DL-th DL_size and i_Out BS(s), generate and load datasets, where i_DL 0-16, i_Out 0-3
def getLoader(i_DL,i_Out):
    in_train_pre,out_train_pre,in_test_pre,out_test_pre,test_index = eachInOutPre(i_DL,num_train,num_test,num_total_user,in_set,out_set)
    in_train = arrangeInput(in_train_pre)
    in_test = arrangeInput(in_test_pre)
    out_train = divideOutput(i_Out,num_beam,out_train_pre)
    out_test = divideOutput(i_Out,num_beam,out_test_pre)

    train_generator = DataSetGenerator(input_set = in_train, output_set = out_train)
    train_loader = data.DataLoader(dataset=train_generator, batch_size=100,shuffle=True)
    
    test_generator = DataSetGenerator(input_set = in_test, output_set = out_test)
    test_loader = data.DataLoader(dataset=test_generator, batch_size=10,shuffle=True)
    return train_loader,test_loader,in_test,out_test
##train_loader,test_loader,in_test,out_test = getLoader(12,2)

In [ ]:
#train and test the network, save the best net in best_net[17][4]
best_net = {}
for i_DL in range(len(DL_size_list)):
    for i_Out in range(0,num_BS,1):
        train_loader,test_loader,in_test,out_test = getLoader(i_DL,i_Out)
        test_loss=[]#average test loss of an epoch
        min_loss=10000#a big number
        for epoch in range(max_epoch):
            
            for batch_idx,(x,label) in enumerate(train_loader):
                x = x.to(device)
                label = label.to(device)
                optimizer.zero_grad()
                y_hat = net(x)
                loss = criterion(label.to(torch.float32),y_hat.to(torch.float32))
                loss.backward()
                optimizer.step()

            for batch_idx,(x,label) in enumerate(test_loader):
                with torch.no_grad():
                    x = x.to(device)
                    label = label.to(device)
                    optimizer.zero_grad()
                    y_hat = net(x)
                    test_loss.append(1e4*criterion(label.to(torch.float32),y_hat.to(torch.float32)))
            batch_loss = sum(test_loss)/len(test_loss)
            if (epoch > min_epoch)and(batch_loss < min_loss):
                min_loss = batch_loss
                print('Min_loss is '+str(min_loss)+'*1e-4 at Epoch '+str(epoch))
                best_net = copy.deepcopy(net.state_dict())
                
        saveNet = 'DLCB_code_output/cache/DL'+str(i_DL)+'Out'+str(i_Out)+'.pth'
        torch.save(best_net,saveNet)                
        if epoch==(max_epoch-1):
            print('End Train and Test of DL size {} and BS {} successfully.'.format(DL_size_list[i_DL],i_Out+1))

In [ ]:
#store DL_Result and save in .mat file
DL_Result={}
for i_DL in range(len(DL_size_list)):
    y_pred = np.zeros((num_test[i_DL],num_beam))
    for i_Out in range(0,num_BS,1):
        _,test_loader,_,out_test = getLoader(i_DL,i_Out)
        saveNet = 'DLCB_code_output/cache/DL'+str(i_DL)+'Out'+str(i_Out)+'.pth'
        net.load_state_dict(torch.load(saveNet,map_location='cuda:0'))
        net.to(device)
        net.eval()
        for batch_idx,(x,label) in enumerate(test_loader):
            with torch.no_grad():
                y_hat = net(x)
                y_hat_np = y_hat.cpu().numpy()
                for i in range(y_hat_np.shape[0]):
                    y_pred[batch_idx*10+i] = y_hat_np[i]
        DL_Result['TX'+str(i_Out+1)+'Pred_Beams'] = y_pred[:,:]
        DL_Result['TX'+str(i_Out+1)+'Opt_Beams'] = out_test[:,:]
    _,_,_,_,test_index = eachInOutPre(i_DL,num_train,num_test,num_total_user,in_set,out_set)
    DL_Result['user_index'] = test_index
    savemat('DLCB_code_output/DL_Result'+str(i_DL+1)+'.mat',DL_Result)
    print('Savemat succeeds.\n')
    #check DL_Result
    print(len(DL_Result['user_index']),DL_Result['TX2Pred_Beams'].shape,DL_Result['TX2Opt_Beams'].shape)